In [1]:
from pathlib import Path
import json
from collections import defaultdict
import shutil
import subprocess

In [2]:
from yolo_scripted import YoloFacade
from utils.general import non_max_suppression, check_img_size, scale_coords, coco80_to_coco91_class
from utils.datasets import letterbox

In [3]:
import numpy as np
import mean_average_precision
import torch
import cv2

In [4]:
from typing import Iterable, List, Any, Callable, Iterable

## one-line scriptng

In [42]:
def script_weights(weights_path, device):
    print(subprocess.Popen(f'python models/export.py --weights {weights_path.as_posix()} --device cuda', 
                           shell=True, 
                           stdout=subprocess.PIPE).stdout.read())
    
    traced_path = weights_path.parents[0] / (weights_path.stem + '.torchscript.pt')
    scripted_path = weights_path.parents[0] / ('scripted_' + weights_path.stem + '.torchscript.pt')
    
    YoloFacade.script(
        yolo_path=weights_path,
        traced_path=traced_path,
        scripted_path=scripted_path,
        device=device
    )
    print(f'scripted model at {scripted_path}')

In [43]:
device = torch.device('cuda')
script_weights(Path('/home/nikita/YOLOv3/evonets_yolo/runs/train/exp25/weights/best.pt'), device)

b"Namespace(batch_size=1, device='cuda', img_size=[640, 640], weights='/home/nikita/YOLOv3/evonets_yolo/runs/train/exp25/weights/best.pt')\nFusing layers... \n\nStarting TorchScript export with torch 1.7.1...\nTorchScript export success, saved as /home/nikita/YOLOv3/evonets_yolo/runs/train/exp25/weights/best.torchscript.pt\nONNX export failure: No module named 'onnx'\nCoreML export failure: No module named 'coremltools'\n\nExport complete (3.65s). Visualize with https://github.com/lutzroeder/netron.\n"
Fusing layers... 
scripted model at /home/nikita/YOLOv3/evonets_yolo/runs/train/exp25/weights/scripted_best.torchscript.pt


## data preprocess

#### merge datasets

In [10]:
old_data_folder = Path('/home/nikita/evonets/data/youtube_alpha')

In [11]:
new_data_folder = Path('/home/nikita/YOLOv3/evonets_yolo/raw_evo_dataset/annotations')

In [12]:
all_images_folder = Path('/home/nikita/images')

In [13]:
with open((old_data_folder / 'params.json').as_posix()) as f:
    old_params = json.load(f)

In [15]:
old_classes = old_params['classes']

In [16]:
# find new class labels

# new_classes = defaultdict(set)
new_classes = old_classes

In [52]:
for filename in new_data_folder.glob('*'):
    with open(filename) as f:
        data = json.load(f)
        for item in data['categories']:
            new_classes[item['name']].add(item['id'])

In [54]:
new_classes = {key: list(value)[0] + 3 for key, value in new_classes.items()}

In [55]:
new_classes.update(old_params['classes'])

In [57]:
# write params new class labels

# with open((old_data_folder / 'params.json').as_posix(), 'w') as f:
#    old_params['classes'] = new_classes
#    json.dump(old_params, f)

In [18]:
new_classes_inverse = {value: key for key, value in new_classes.items()}

In [23]:
for key in sorted(new_classes_inverse.keys()):
    print(key, new_classes_inverse[key])

1 road
2 автомобиль
3 пешеход
4 Главная дорога
5 Уступи дорогу
6 Стоп
7 Кирпич
8 Светофор
9 Красный треугольник
10 Красный круг
11 Синий круг
12 Синий квадрат
13 Табличка
14 Другие знаки


get markdown from new data

In [37]:
image_name_to_annotations = defaultdict(list)
image_name_to_info = {}

for filename in new_data_folder.glob('*'):
    image_id_to_image_name = {}
    
    with open(filename) as f:
        data = json.load(f)
        for item in data['images']:
            image_id_to_image_name[item['id']] = Path(item['file_name']).name
            image_name_to_info[Path(item['file_name']).name] = item
            
        
        for item in data['annotations']:
            detection = {'label': new_classes_inverse[item['category_id']], 'bbox': item['bbox']}
            image_name_to_annotations[image_id_to_image_name[item['image_id']]].append(detection)

In [40]:
image_name_to_annotations

defaultdict(list,
            {'4T3dDO4Rmrk_044880.jpg': [{'label': 'Красный круг',
               'bbox': [1788.65, 548.73, 21.6, 15.33]},
              {'label': 'Кирпич', 'bbox': [1787.95, 517.02, 27.88, 30.31]}],
             '4T3dDO4Rmrk_044820.jpg': [{'label': 'Синий круг',
               'bbox': [203.82, 539.83, 22.16, 48.51]}],
             '4T3dDO4Rmrk_044640.jpg': [{'label': 'Синий круг',
               'bbox': [281.67, 575.42, 25.79, 47.39]}],
             '4T3dDO4Rmrk_044580.jpg': [{'label': 'Синий круг',
               'bbox': [567.63, 620.19, 22.07, 30.2]},
              {'label': 'Синий круг',
               'bbox': [1763.89, 347.71, 156.11, 148.72]}],
             '4T3dDO4Rmrk_044520.jpg': [{'label': 'Красный треугольник',
               'bbox': [1852.31, 319.45, 36.58, 38.62]},
              {'label': 'Красный круг',
               'bbox': [1854.63, 359.52, 35.72, 20.91]}],
             '4T3dDO4Rmrk_044280.jpg': [{'label': 'Синий круг',
               'bbox': [1442.72,

In [43]:
image_name_to_info

{'4T3dDO4Rmrk_045240.jpg': {'id': 1,
  'width': 1920,
  'height': 1080,
  'file_name': 'splitted/4T3dDO4Rmrk_00/4T3dDO4Rmrk_045240.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 '4T3dDO4Rmrk_045180.jpg': {'id': 2,
  'width': 1920,
  'height': 1080,
  'file_name': 'splitted/4T3dDO4Rmrk_00/4T3dDO4Rmrk_045180.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 '4T3dDO4Rmrk_045120.jpg': {'id': 3,
  'width': 1920,
  'height': 1080,
  'file_name': 'splitted/4T3dDO4Rmrk_00/4T3dDO4Rmrk_045120.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 '4T3dDO4Rmrk_045060.jpg': {'id': 4,
  'width': 1920,
  'height': 1080,
  'file_name': 'splitted/4T3dDO4Rmrk_00/4T3dDO4Rmrk_045060.jpg',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0},
 '4T3dDO4Rmrk_045000.jpg': {'id': 5,
  'width': 1920,
  'height': 1080,
  'file_name': 'splitted/4T3dDO4Rmrk_00/4T3dDO4Rmrk_045000.jpg',
  'lice

add new data to existing dataset

In [24]:
annotations_folder = old_data_folder / 'annotations'
images_folder = old_data_folder / 'images'

In [46]:
for image_filename, detections in image_name_to_annotations.items():
    an_file = annotations_folder / (Path(image_filename).stem + '.json')
    
    if an_file.exists():
        with open(an_file) as f:
            data = json.load(f)

        data['objects'] = data['objects'] + detections
        
    else:
        shutil.copy(all_images_folder / image_filename, images_folder / image_filename)
        
        data = {}
        data['width'] = image_name_to_info[image_filename]['width']
        data['height'] = image_name_to_info[image_filename]['height']
        data['filename'] = image_filename
        data['objects'] = detections
    
    with open(an_file, 'w') as f:
        json.dump(data, f)

#### make training dataset

In [25]:
dataset_path = Path('/home/nikita/YOLOv3/evonets_yolo/evo_dataset')

In [26]:
def objects_to_strings(objects, width, height):
    result = []
    for obj in objects:
        if 'bbox' not in obj:
            continue
        
        label = new_classes[obj['label']]
        
        x = (obj['bbox'][0] + obj['bbox'][2] / 2) / width
        y = (obj['bbox'][1] + obj['bbox'][3] / 2) / height
        w = obj['bbox'][2] / width
        h = obj['bbox'][3] / height
        
        result.append(f'{label} {x} {y} {w} {h}\n')
        
    return result

In [27]:
def objects_to_gt(objects):
    result = []
    
    for obj in objects:
        if 'bbox' not in obj:
            continue
            
        label = new_classes[obj['label']]
        bbox = obj['bbox']
        result.append(bbox + [label])
        
    return result

In [28]:
im_train_dir = dataset_path / 'images' / 'train'
im_val_dir = dataset_path / 'images' / 'val'
im_test_dir = dataset_path / 'images' / 'test'
an_train_dir = dataset_path / 'labels' / 'train'
an_val_dir = dataset_path / 'labels' / 'val'
an_test_dir = dataset_path / 'labels' / 'test'

In [29]:
test_size = 2000
val_size = 500

In [67]:
for index, filename in enumerate(annotations_folder.glob('*.json')):    
    with open(filename) as f:
        data = json.load(f)
        
        stem = Path(data['filename']).stem
        
        image_path = images_folder / data['filename']
        width = data['width']
        height = data['height']
        objects = data['objects']
        
        new_annotations = objects_to_strings(objects, width, height)
        
        if index < test_size:
            im_dir = im_test_dir
            an_dir = an_test_dir
        elif test_size + val_size > index >= test_size:
            im_dir = im_val_dir
            an_dir = an_val_dir
        else:
            im_dir = im_train_dir
            an_dir = an_train_dir
            
        with open(an_dir / (stem + '.txt'), 'w') as f:
            f.writelines(new_annotations)
                
        shutil.copy(image_path, im_dir / (stem + '.jpg'))

#### now i manually write evo_dataset.yaml

### train

In [47]:
!python train.py --img 640 --batch 2 --epochs 1 --data data/coco128.yaml --weights yolov3-spp.pt

Using torch 1.7.1 CUDA:0 (GeForce RTX 2070 SUPER, 7979MB)

Namespace(adam=False, batch_size=2, bucket='', cache_images=False, cfg='', data='data/coco128.yaml', device='', epochs=1, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=-1, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/exp26', single_cls=False, sync_bn=False, total_batch_size=2, weights='yolov3-spp.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.

## test

In [30]:
# abstract

def get_mAP(predicted, groundtruth, n_classes, classes_mapping=None):
    """Evaluates detection metric.
    
    Args:
        predicted: [images X detections on image X [x_left y_top x_right y_bottom confidence label]]
        groundtruth: [images X detections on image X [x_left y_top width height label]]
        n_classes: number of classes
        classes_mappng: from predicted to groundtruth. For example, groundtruth has label '3' for person,
            predicted has label '2' for person. Then pass classes_mapping={2: 3}
            
    Returns:
        dict: {'COCO_mAP': mAP@0.5:0.95:0.05,
               'PASCAL_mAP': mAP@0.5,
               'class_AP': {class label: AP@0.5}}
    """    
    metric_builder = mean_average_precision.MeanAveragePrecision2d(n_classes)
    
    for one_image_pred, one_image_gt in zip(predicted, groundtruth):
        one_image_gt = np.array(one_image_gt)
        one_image_pred = np.array(one_image_pred)
        
        if one_image_gt.size == 0:
            continue
        
        # [xmin, ymin, xmax, ymax, label, confidence]
        one_image_pred = one_image_pred[:, [0, 1, 2, 3, 5, 4]]
        for detection in one_image_pred:            
            # map labels
            
            if classes_mapping is not None and detection[5] in classes_mapping:
                detection[5] = classes_mapping[detection[5]]
                
        # [xmin, ymin, xmax, ymax, class_id, difficult, crowd]
        for detection in one_image_gt:
            #xywh -> xyxy
            detection[2] += detection[0]
            detection[3] += detection[1]
            
            
        # difficult, crowd
        gt = np.zeros((one_image_gt.shape[0], 7), dtype=int)
        gt[:one_image_gt.shape[0],:one_image_gt.shape[1]] = one_image_gt
          
        metric_builder.add(one_image_pred, gt)
        
    coco_metrics = metric_builder.value(iou_thresholds=np.arange(0.5, 1.0, 0.05), 
                                    recall_thresholds=np.arange(0., 1.01, 0.01), 
                                    mpolicy='soft')
    
    ap = {key: value['ap'] for key, value in coco_metrics[0.5].items()}
    
    coco_map = coco_metrics['mAP']
    
    pascal_map = metric_builder.value(iou_thresholds=0.5, 
                                      recall_thresholds=np.arange(0., 1.1, 0.1))['mAP']
    
    
    
    return {'COCO_mAP': coco_map, 'PASCAL_mAP': pascal_map, 'class_AP': ap}
    

In [31]:
def detect_single(source: Path, model, device: torch.device, imgsz: int = 640):
        image = source.absolute().as_posix()

        max_stride = model.stride.max()
        imgsz = check_img_size(imgsz, s=max_stride)  # check img_size

        # Run inference
        im0 = cv2.imread(image, 1)
        img = letterbox(im0, new_shape=imgsz)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        img = np.ascontiguousarray(img)

        img = torch.from_numpy(img).to(device)
        img = img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        with torch.no_grad():
            pred = model(img)

        # Process detections
        det = pred[0]
        if len(det) > 0:
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape)

        return det


#### base

In [32]:
base_path = Path('/home/nikita/YOLOv3/evonets_yolo/whole-yolov3-spp.torchscript.pt')

In [105]:
model = YoloFacade.load(base_path, device)

In [106]:
groundtruth = []
predicted = []

for filename in an_test_dir.glob('*'): 
    filename = annotations_folder / (filename.stem + '.json')
    with open(filename) as f:
        data = json.load(f)

        objects = data['objects']
        
        groundtruth.append(objects_to_gt(objects))
        
    image = im_test_dir / (filename.stem + '.jpg')
    predicted.append(detect_single(image, model, device).cpu().numpy())

In [107]:
mapping = {i: 0 for i in range(100)}
mapping[0] = 13  # person
mapping[2] = 12  # automobile
mapping[5] = 12  # bus
mapping[7] = 12  # truck

In [108]:
get_mAP(predicted, groundtruth, 15, mapping)

{'COCO_mAP': 0.022142084,
 'PASCAL_mAP': 0.03732616,
 'class_AP': {0: 0.0,
  1: 0.0,
  2: 0.49696103,
  3: 0.00046410892,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 3.0371136e-06,
  8: 0.0,
  9: 0.028071586,
  10: 0.003150315,
  11: 0.00054853864,
  12: 0.0,
  13: 0.0,
  14: 0.0}}

#### new

In [35]:
script_weights(Path('/home/nikita/YOLOv3/evonets_yolo/runs/train/exp21/weights/best.pt'), device)

b"Namespace(batch_size=1, device='cuda', img_size=[640, 640], weights='/home/nikita/YOLOv3/evonets_yolo/runs/train/exp21/weights/best.pt')\nFusing layers... \n\nStarting TorchScript export with torch 1.7.1...\nTorchScript export success, saved as /home/nikita/YOLOv3/evonets_yolo/runs/train/exp21/weights/best.torchscript.pt\nONNX export failure: No module named 'onnx'\nCoreML export failure: No module named 'coremltools'\n\nExport complete (2.88s). Visualize with https://github.com/lutzroeder/netron.\n"
Fusing layers... 
scripted model at /home/nikita/YOLOv3/evonets_yolo/runs/train/exp21/weights/scripted_best.torchscript.pt


In [36]:
new_path = Path('/home/nikita/YOLOv3/evonets_yolo/runs/train/exp21/weights/scripted_best.torchscript.pt')

In [37]:
model = YoloFacade.load(new_path, device)

In [38]:
groundtruth = []
predicted = []

for filename in an_test_dir.glob('*'): 
    filename = annotations_folder / (filename.stem + '.json')
    with open(filename) as f:
        data = json.load(f)

        objects = data['objects']
        
        groundtruth.append(objects_to_gt(objects))
        
    image = im_test_dir / (filename.stem + '.jpg')
    predicted.append(detect_single(image, model, device).cpu().numpy())

In [40]:
get_mAP(predicted, groundtruth, 15)

{'COCO_mAP': 0.2274824,
 'PASCAL_mAP': 0.40620223,
 'class_AP': {0: 0.0,
  1: 0.62879145,
  2: 0.8526745,
  3: 0.6456506,
  4: 0.0,
  5: 0.48287007,
  6: 0.6622555,
  7: 0.7083287,
  8: 0.5269795,
  9: 0.68066263,
  10: 0.486411,
  11: 0.5135398,
  12: 0.0,
  13: 0.0,
  14: 0.0}}